In [1]:
import tensorflow as tf
import numpy as np

2023-07-29 20:46:35.785793: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-29 20:46:36.247606: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-29 20:46:38.081891: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# %load_ext tensorboard
# %tensorboard --logdir runs

In [3]:
batch_size = 4

In [4]:
def data_gen(file_names):
    for file_name in file_names:
        with open(file_name, 'r') as f:
            xs, ys = [], []
            for line in f:
                line = line.strip()
                if not line:
                    yield (xs, ys)
                    xs, ys = [], []
                else:
                    word, tag = line.split()
                    xs.append(word)
                    ys.append(tag)

dataset = tf.data.Dataset.from_generator(
    data_gen,
    args = [("eng.train.iob", "eng.testa.iob", "eng.testb.iob")],
    output_signature= (
        tf.TensorSpec(shape=(None,), dtype=tf.string),
        tf.TensorSpec(shape=(None,), dtype=tf.string)
    )
)



# val_dataset = tf.data.Dataset.from_generator(
#     data_gen,
#     args = ["eng.testa.iob"],
#     output_signature= (
#         tf.TensorSpec(shape=(None,), dtype=tf.string),
#         tf.TensorSpec(shape=(None,), dtype=tf.string)
#     ))

sentences = dataset.map(lambda xs, _ : xs)
tags = dataset.map(lambda _, ys : ys)
layer_sen = tf.keras.layers.TextVectorization(standardize="lower_and_strip_punctuation")
layer_tag = tf.keras.layers.TextVectorization(standardize="lower_and_strip_punctuation", max_tokens=8)
layer_sen.adapt(sentences)
layer_tag.adapt(tags)
# print(layer_sen.get_vocabulary())
vocab_size = layer_sen.vocabulary_size()
# print(vocab_size)
num_tags = layer_tag.vocabulary_size()

2023-07-29 20:46:41.073928: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-07-29 20:46:41.465426: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-07-29 20:46:41.465478: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-07-29 20:46:41.470374: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-07-29 20:46:41.470429: I tensorflow/compile

In [5]:
dataset = dataset.shuffle(1000, reshuffle_each_iteration=True)

In [6]:
vocab = layer_tag.get_vocabulary()
vocab

['', '[UNK]', 'o', 'ius', 'iaa', 'bus', 'baa']

In [7]:
d = dataset.map(lambda xs, ys: (xs, ys))
data = dataset.map(lambda xs, ys: (tf.reshape(layer_sen(xs), tf.shape(xs)), tf.reshape(layer_tag(ys), tf.shape(ys)))).padded_batch(batch_size)
# validation_dataset = val_dataset.map(lambda xs, ys: (tf.reshape(layer_sen(xs), tf.shape(xs)), tf.reshape(layer_tag(ys), tf.shape(ys)))).padded_batch(batch_size)
# list(train_dataset.take(2))

In [8]:
DATASET_SIZE = len(list(data))
TRAIN_SIZE = int(0.8 * DATASET_SIZE)
VAL_SIZE = DATASET_SIZE - TRAIN_SIZE

train_data = data.take(TRAIN_SIZE)
val_data = data.skip(VAL_SIZE)

2023-07-29 20:46:51.103899: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_10' with dtype int64
	 [[{{node Placeholder/_10}}]]
2023-07-29 20:46:51.104245: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_15' with dtype string
	 [[{{node Placeholder/_15}}]]


In [10]:
class EncoderLSTM(tf.keras.layers.Layer):

    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        super(EncoderLSTM, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim, mask_zero=True)
        self.lstm = tf.keras.layers.LSTM(self.enc_units, return_sequences=True)

    def call(self, x):
        x = self.embedding(x)
        output = self.lstm(x)

        return output
    

class DecoderLSTM(tf.keras.layers.Layer):

    def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
        super(DecoderLSTM, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim, mask_zero=True)
        self.lstm = tf.keras.layers.LSTM(self.dec_units, return_sequences=True)
        self.fc = tf.keras.layers.Dense(vocab_size)

    def call(self, x, hidden):
        x = self.embedding(x)
        output = self.lstm(x, initial_state = hidden)
        output = self.fc(output)

        return output
                                                   

class MyModel(tf.keras.Model):
    def __init__(self, vocab_size=vocab_size, num_tags=num_tags):
        super().__init__()
        # self.minput = tf.keras.Input(type_spec=tf.TensorSpec(shape=(None, None), dtype=tf.int32))
        self.embedding = tf.keras.layers.Embedding(vocab_size+1, 32, mask_zero=True)
        self.lstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=True))
        self.layer_norm = tf.keras.layers.LayerNormalization()
        # self.dropout1 = tf.keras.layers.Dropout(0.5)
        self.dense1 = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(16, activation='relu'))
        # self.dropout2 = tf.keras.layers.Dropout(0.5)
        self.dense2 = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(num_tags, activation="softmax"))

    def call(self, inputs):
        # x = self.minput(inputs)
        x = self.embedding(inputs)
        x = self.lstm(x)
        x = self.layer_norm(x)
        # x = self.dropout1(x)
        x = self.dense1(x)
        # x = self.dropout2(x)
        x = self.dense2(x)
        return x

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = nll(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)



optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
nll = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)

model = MyModel(vocab_size=vocab_size, num_tags=num_tags)
model.compile(loss=loss_function, optimizer=optimizer)
model.fit(train_data, epochs=20, validation_data=val_data)

Epoch 1/20


2023-07-29 20:48:59.217368: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/ReverseV2_grad/ReverseV2/ReverseV2/axis' with dtype int32 and shape [1]
	 [[{{node gradients/ReverseV2_grad/ReverseV2/ReverseV2/axis}}]]
2023-07-29 20:49:01.809806: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/ReverseV2_grad/ReverseV2/ReverseV2/axis' with dtype int32 and shape [1]
	 [[{{node gradients/ReverseV2_grad/ReverseV2/ReverseV2/axis}}]]
2023-07-29 20:49:03.001325: W tensorflow/core/common_runtime/type_inference.cc:339] Type inference failed. This indicates an invalid graph that escaped type checking. Error message

    515/Unknown - 128s 226ms/step - loss: 0.4385

2023-07-29 20:51:05.431122: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_11' with dtype string
	 [[{{node Placeholder/_11}}]]
2023-07-29 20:51:05.431504: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_14' with dtype int64
	 [[{{node Placeholder/_14}}]]


516/516 [==============================] - 144s 256ms/step - loss: 0.4384 - val_loss: 0.2932
Epoch 2/20
516/516 [==============================] - 105s 203ms/step - loss: 0.2807 - val_loss: 0.2200
Epoch 3/20
516/516 [==============================] - 114s 222ms/step - loss: 0.2264 - val_loss: 0.1782
Epoch 4/20
516/516 [==============================] - 111s 210ms/step - loss: 0.1855 - val_loss: 0.1571
Epoch 5/20
516/516 [==============================] - 108s 209ms/step - loss: 0.1609 - val_loss: 0.1412
Epoch 6/20
516/516 [==============================] - 102s 197ms/step - loss: 0.1521 - val_loss: 0.1305
Epoch 7/20
516/516 [==============================] - 104s 200ms/step - loss: 0.1349 - val_loss: 0.1174
Epoch 8/20
516/516 [==============================] - 104s 201ms/step - loss: 0.1280 - val_loss: 0.1326
Epoch 9/20
516/516 [==============================] - 107s 208ms/step - loss: 0.1063 - val_loss: 0.1049
Epoch 10/20
516/516 [==============================] - 98s 190ms/step - los

In [25]:
predictions = model.predict(layer_sen("Unable to print".split()))
# predictions = model.predict(tf.constant([[282, 12]]))
# tf.keras.activations.softmax(predictions)
tf.argmax(predictions, axis=-1)
# tf.keras.activations.softmax(predictions)

1/1 [==============================] - 0s 46ms/step


<tf.Tensor: shape=(3, 1), dtype=int64, numpy=
array([[5],
       [6],
       [5]])>

In [26]:
vocab = layer_tag.get_vocabulary()
" ".join([vocab[each] for each in tf.squeeze(tf.argmax(predictions, axis=-1))])

'bus baa bus'